In [15]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sc
import matplotlib.pyplot as plt
import yaml


from models.model2 import BicycleModel
from models.model4 import FourWheelModel
from models.utils import get_csv_row_count
from models.utils import get_folder_path
from models.utils import fit_circle
from models.utils import plot_column_histograms

In [159]:


class ParticleBasedMethod:
    def __init__(self, num_particles):
        self.num_particles = num_particles
        self.list_of_param = ['T_peak', 'T_slope']
        self.param_to_infer = len(self.list_of_param)
        self.variances_to_infer = 1

        self.prior_mean = 0.5
        self.prior_stddev = 0.1


        self.particles = None
        self.weights = None
        self.initialize_particles()

        self.total_particles = self.particles


                # Current model and model parameters
        self.model = None
        self.dt = 0.025
        self.commands = get_folder_path()+'/open_loop_inputs/open_loop_commands.csv'
        self.N = get_csv_row_count(self.commands)
        self.open_loop_tf = self.N*self.dt
        # T_peak and T_slope is not specified since we infer those
        self.x_vect_ref = self.get_x_vect_ref()
        self.y_likelihood = 0



    def initialize_particles(self):
        self.particles = np.zeros((self.num_particles, self.param_to_infer+self.variances_to_infer))
        self.weights = np.ones(self.num_particles) / self.num_particles
        for i in range(self.num_particles):
            new = self.prior_sampler()
            for j in range(self.particles[i].size):
                self.particles[i][j] = new[j]

    def update_particles(self):
        for i in range(self.num_particles):
            self.particles[i] = self.proposal_sampler(self.particles[i])
            #print( 'Likelihood:', self.likelihood_evaluator(self.particles[i]))
            self.weights[i] *= self.likelihood_evaluator(self.particles[i])
        for i in range(self.weights):
            
        self.weights /= np.sum(self.weights)
        #print('Weights: ', self.weights)

    def resample_particles(self):
        indices = np.random.choice(range(self.num_particles), size=self.num_particles, replace=True, p=self.weights) 
        self.particles = self.particles[indices]
        self.weights = np.ones(self.num_particles) / self.num_particles


    # Define the prior distribution sampler
    def prior_sampler(self):
        # Implement your own prior distribution sampling logic
        # Return a sample from the prior distribution                
        sample = np.random.normal(self.prior_mean, self.prior_stddev, size= self.param_to_infer+self.variances_to_infer)
        return sample

    # Define the proposal distribution sampler
    def proposal_sampler(self, particle):
        proposal_mean = particle  # Use the current particle as the mean for the proposal distribution
        proposal_stddev = 0.1  # Choose an appropriate standard deviation for the proposal distribution
        sample = np.random.normal(proposal_mean, proposal_stddev, size=len(particle))
        return sample

    # Define the likelihood evaluator
    def likelihood_evaluator(self, particle):
        
        # Given the observed data and a particle, compute the likelihood
        predicted_kpi = self.get_kpi(particle)  # Replace `model_simulation` with your own function that generates predicted data
        # Compute the likelihood using a probability distribution or a statistical measure
        if self.variances_to_infer == 1:
            log_likelihood_vect = sc.norm.logpdf(self.y_likelihood, loc= predicted_kpi, scale=particle[self.param_to_infer]) # Loc will be simulation(theta(:param_to_infer), scale = theta[param_to_infer])
            #print('log_likelihood_vect', log_likelihood_vect)
        else:
            mean_vector = np.full(self.param_to_infer, predicted_kpi)
            log_likelihood_vect = sc.multivariate_normal.logpdf(self.y_likelihood, mean= mean_vector, cov=np.diag(particle[self.param_to_infer:])) # Loc will be simulation(theta(:param_to_infer), scale = theta[param_to_infer])
        log_likelihood_out = np.sum(log_likelihood_vect)

        likelihood = np.exp(log_likelihood_out)
        
        return likelihood


    
    #--------------- Open loop simulation ---------------#

    def generate_new_model(self, Tetha):
        T_peak = Tetha[0]
        T_slope = Tetha[1]
        self.model = FourWheelModel( self.dt, self.open_loop_tf,float(T_peak), float(T_slope))
    
    def get_kpi(self, particle):
        self.generate_new_model(particle[:self.param_to_infer])
        
        t,x_vect = self.get_open_loop_data()
        
        l2_norm_position = np.linalg.norm(self.x_vect_ref[:,:2] - x_vect[:,:2], axis=1).sum()
        l2_norm_velocity = np.linalg.norm(self.x_vect_ref[:,2:] - x_vect[:,2:], axis=1).sum()

        w1 = 0.00005
        w2 = 0.00005
        kpi = w1*l2_norm_position+w2*l2_norm_velocity
        return kpi


    def get_open_loop_data(self):
        
        t,x_vect = self.model.do_open_loop_sim_from_csv(self.commands)
        
        return t,x_vect
    

    
    def get_x_vect_ref(self):
        T_peak_ref = 0.37
        T_slope_ref = 0.4
        self.model = FourWheelModel(self.dt,self.open_loop_tf, T_peak_ref, T_slope_ref)
        t,x = self.get_open_loop_data()
        return x


    #
    def run_inference(self, num_iterations):
        for i in range(num_iterations):
            self.update_particles()
            self.resample_particles()

            print('Iteration: ',i)

        return self.particles
    
        

# Instantiate the particle-based method
num_particles = 20  # Number of particles to use
particle_method = ParticleBasedMethod(num_particles)

# Run the inference
num_iterations = 150  # Number of iterations
particles = particle_method.run_inference(num_iterations)


Iteration:  0
Iteration:  1
Iteration:  2
Iteration:  3
Iteration:  4
Iteration:  5
Iteration:  6
Iteration:  7
Iteration:  8
Iteration:  9
Iteration:  10
Iteration:  11
Iteration:  12
Iteration:  13
Iteration:  14
Iteration:  15
Iteration:  16
Iteration:  17
Iteration:  18
Iteration:  19
Iteration:  20
Iteration:  21
Iteration:  22
Iteration:  23
Iteration:  24
Iteration:  25
Iteration:  26
Iteration:  27
Iteration:  28
Iteration:  29
Iteration:  30
Iteration:  31
Iteration:  32
Iteration:  33
Iteration:  34
Iteration:  35
Iteration:  36
Iteration:  37
Iteration:  38
Iteration:  39
Iteration:  40
Iteration:  41
Iteration:  42
Iteration:  43
Iteration:  44
Iteration:  45
Iteration:  46
Iteration:  47
Iteration:  48
Iteration:  49
Iteration:  50
Iteration:  51
Iteration:  52
Iteration:  53
Iteration:  54
Iteration:  55
Iteration:  56
Iteration:  57
Iteration:  58
Iteration:  59
Iteration:  60
Iteration:  61
Iteration:  62
Iteration:  63
Iteration:  64
Iteration:  65
Iteration:  66
Itera

ValueError: probabilities contain NaN

In [160]:
particle_method.particles

array([[ 0.52467911,  0.59336553,  1.19187719],
       [ 0.4416403 ,  0.21531031,  0.60483516],
       [ 0.23652595,  1.3779604 ,  0.76577217],
       [ 0.2017965 ,  0.63669423,  0.6336348 ],
       [ 0.50914222,  0.36589511,  0.83957029],
       [ 0.52409109,  0.21194425,  0.98195148],
       [ 0.51599686,  0.48764474,  0.86500224],
       [ 0.50239275,  0.60998877,  0.38745247],
       [ 0.54548736,  0.37976831,  0.79728799],
       [ 0.5328461 ,  0.57422213,  0.66613392],
       [ 0.37658382,  0.27530328,  0.78388226],
       [ 0.76017233, -0.05367715,  0.96297044],
       [ 0.28903256,  0.53615125,  0.58794414],
       [ 0.14026249,  0.21947154,  0.75995998],
       [ 0.53834963,  0.41013575,  1.18661378],
       [ 0.30804951,  0.45380694,  0.57145236],
       [ 0.28130621,  0.44703435,  0.92295298],
       [ 0.43317666,  0.39592901,  0.52589695],
       [ 0.19725247,  0.54489557,  0.7958609 ],
       [ 0.56630437,  0.3175405 ,  0.93060219]])